In [0]:
from pyspark.sql import functions as F

In [0]:
# Configurações do Data Lake
storage_account = "storagevinicius"
container = "datalake"

bronze_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/sp500_prices_delta"
silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/sp500_prices"

In [0]:
df = spark.read.format("delta").load(bronze_path)

df = df.withColumn("date", F.to_date(F.col("date")))

null_dates = df.filter(F.col("date").isNull()).count()
total_rows = df.count()

print("Total linhas:", total_rows)
print("Datas nulas após to_date:", null_dates)

In [0]:
df_silver = (
    df
    .filter(F.col("date").isNotNull())
    .filter(F.col("date") >= F.lit("2014-01-01"))
    .orderBy("date")
)

print("Linhas após filtro (>= 2014):", df_silver.count())

In [0]:
(
    df_silver.write
    .format("delta")
    .mode("overwrite")
    .save(silver_path)
)

print("Silver salvo em:", silver_path)

In [0]:
df_check = spark.read.format("delta").load(silver_path)
print("Linhas no Silver:", df_check.count())
print("Período (min/max):")
display(df_check.select(F.min("date").alias("min_date"), F.max("date").alias("max_date")))